# 원하는 검색어로 네이버 뉴스만 크롤링

In [4]:
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver

In [8]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수
# 입력된 수를 1,11,21,31 ... 로 만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0 :
        return num+1
    else:
        return num+9*(num-1)
    
# 크롤링 할 url 생성하는 함수 만들기( 검색어, 크롤링 시작 페이지, 크롤링 종료페이지)

def makeUrl(search,start_pg,end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url:",url)
        return url

    else:
        urls = []
        for i in range(start_pg,end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start" + str(page)
            urls.append(url)
        print("생성url: ",urls)
        return urls


In [12]:
# 검색어를 입력받아 네이버 검색 결과 페이지 url 생성

# 검색어 입력
search = input("검색할 키워드를 입력해주세요:")

# 검색 시작할 페이지 입력
page = int(input('\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):'))
print("\n크롤링할 시작 페이지: ",page,'페이지')

# 검색 종료할 페이지 입력
page2 = int(input('\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):'))
print("\n크롤링할 종료 페이지: ",page2,'페이지')

# naver url 생성
search_urls = makeUrl(search,page,page2)

검색할 키워드를 입력해주세요: 코로나

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력): 1



크롤링할 시작 페이지:  1 페이지



크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력): 2



크롤링할 종료 페이지:  2 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start11']


In [13]:
# Selenium 사용하여 네이버 기사 url 추출

# selenium으로 navernews만 뽑아오기
driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(3)

# selenium으로 검색 페이지 불러오기

naver_urls = []
count = 0 # 핸들러 변수(창 바꿀때)

for i in search_urls:
    driver.get(i)
    time.sleep(1) #대기시간 변경 가능
    
    # 네이버 기사 눌러서 제목 및 본문 가져오기
    # 네이버 기사가 있는 기사 css selector 모아오기
    a = driver.find_elements_by_css_selector('a.info')
    
    # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url 얻기
    for i in a:
        count += 1
        i.click()
        
        #현재 탭에 접근
        driver.switch_to.window(driver.window_handles[count-1])
        time.sleep(3) # 대기시간
        
        # 네이버 뉴스 url만 가져오기
        url = driver.current_url
        print(url)
        
        if "news.naver.com" in url:
            naver_urls.append(url)
            
        else:
            pass
        
        # 다시처음 탭으로 돌아가기(매우 중요)
        driver.switch_to.window(driver.window_handles[0])
        
print(naver_urls)

C:\Users\user\AppData\Local\Temp/ipykernel_13868/2140456141.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')
C:\Users\user\AppData\Local\Temp/ipykernel_13868/2140456141.py:18: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  a = driver.find_elements_by_css_selector('a.info')


https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%BD%94%EB%A1%9C%EB%82%98&start1
https://www.yna.co.kr/
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=001&aid=0013010022
https://www.khan.co.kr/
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=032&aid=0003130441
https://star.mt.co.kr/
https://entertain.naver.com/read?oid=108&aid=0003031156
https://newsis.com/
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=003&aid=0011024241
https://www.mt.co.kr/
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=008&aid=0004712568
https://www.edaily.co.kr/
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=018&aid=0005153926
https://biz.chosun.com/
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=366&aid=0000795894
https://www.edaily.co.kr/
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=018&aid=0005154018
https://www.xportsnews.com/
https://ent

In [19]:
# naver 기사 본문 및 제목 가져오기

# ConnectionError방지
headers = {"User-Agent":"Mozilla/5.0 (Window NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

titles = []
contents = []
for i in naver_urls:
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text,"html.parser")
    # 검색결과 확인시
    #print(html)
    
    # 뉴스 제목 가져오기
    title = html.select("div.content > div.article_header > div.article_info > h3")
    # list 합치기
    title = ''.join(str(title))
    # html태그 제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1,repl = '', string=title)
    titles.append(title)
    
    # 뉴스 본문 가져오기
    content = html.select("div.content > div#articleBody > div#articleBodyContents")
    
    # 기사 텍스트만 가져오기
    # list 합치기
    content = ''.join(str(content))
    
    # html태그제거 및 텍스트 다듬기
    content = re.sub(pattern = pattern1,repl = '', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가 \nfunction _flash_removeCallback() {}]"""
    content = content.replace(pattern2,'')
    
    contents.append(content)

print(titles)
print(contents)

['[코로나19 확진돼 재택치료하던 생후 4개월 남아 또 숨져(종합)]', '[대구서 코로나19 확진 7세 여아 숨져···“급성심근염 추정”]', '[씨젠, 30분만에 코로나19 검사 가능한 진단시약 개발]', "[자가진단키트 품귀에 삼성이 또…코로나 뚫는 '이재용의 동행']", '[지하도상가 등 1만개 점포 임대료 최대 60% 감면… 코로나 피해지원]', '[씨젠, 30분 만에 코로나19 검사 가능한 진단시약 개발]', '["코로나 자가검사키트 특별공급"...이런 문자는 사기입니다]', '[삼성전자, 이번엔 코로나 진단키트 ‘해결사’로 나섰다]', '[코로나19 확진돼 재택치료하던 생후 4개월 남아 또 숨져(종합)]', '[대구서 코로나19 확진 7세 여아 숨져···“급성심근염 추정”]', '[씨젠, 30분만에 코로나19 검사 가능한 진단시약 개발]', "[자가진단키트 품귀에 삼성이 또…코로나 뚫는 '이재용의 동행']", '[지하도상가 등 1만개 점포 임대료 최대 60% 감면… 코로나 피해지원]', '["코로나 자가검사키트 특별공급"...이런 문자는 사기입니다]', '[삼성전자, 이번엔 코로나 진단키트 ‘해결사’로 나섰다]']
['[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n신속히 병원 이송됐으나 소생 못 해…18일에는 7개월 영아 사망(수원=연합뉴스) 강영훈 기자 = 신종 코로나바이러스 감염증(코로나19)에 확진돼 재택 치료를 받던 생후 4개월 된 남아가 22일 숨지는 일이 발생했다.코로나19 환자 이송하는 의료진 (CG)본 기사 내용과 관련 없음. [연합뉴스TV 제공]    앞서 지난 18일에는 생후 7개월 된 확진자가 병원 이송 중 사망하는 등 코로나19 영아 환자가 잇달아 숨지면서 재택 치료자 관리에 대한 우려가 커지고 있다.    24일 경찰과 소방당국에 따르면 지난 22일 오후 1시 52분 경기도 수원시 권선구의 생후 4개월 된 A군의 부모로부터 "코로나

In [20]:
# Dataframe으로 만들기

# 데이터프레임으로 정리
import pandas as pd

news_df = pd.DataFrame({'title':titles,'link':naver_urls,'content':contents})

news_df.to_csv('NaverNews_%s.csv'%search,index = False, encoding = 'utf-8=sig')